## Complete Grammar

In [2]:
import string
import os
import json
from fractions import Fraction

def create_grammar():
    print("Creating generic grammar ")
    path = os.path.join('grammars_test','grammar1.json')

    #Setting the options
    name = "16grammar44"
    namespace = "16grammar44"
    meter_nb_beats = 4
    meter_beat_unit = 5
    levels = 4  #how many time non terminal symbols can be transformed in other non terminal
    grace_notes = 1 #maximum number of grace notes allowed
    list_of_divisions = [2,3] #list_of_division must contain only prime numbers. We can add 11, 13, etc. if we need complex tuplets
    nb_parse_fail = 0

    #create the json
    json_grammar = {
        "name": name,
        "namespace": namespace,
        "meter_nb_beats": meter_nb_beats,
        "meter_beat_unit": meter_beat_unit,
        "parse_failures_ratio": nb_parse_fail,
        "states": [],
        "rules": []

    }
    ## Create States
    # Create Terminal States
    json_grammar["states"].append({"name": "N0", "type": "cont", "nb_grace_notes":0})
    for ii in range(grace_notes + 1):
        json_grammar["states"].append(
            {"name": "N{0}".format(ii + 1), "type": "note", "nb_grace_notes": ii})
    ##Create non terminal states and rules with the recursive funcion
    first_state = {"name": (Fraction(0),Fraction(1)), "type": "non-terminal"}
    recursive_output = recursive_state_creation(first_state, list_of_divisions, 1, levels, grace_notes)
    json_grammar["states"].append(first_state)
    json_grammar["states"].extend(recursive_output[0])
    json_grammar["rules"].extend(recursive_output[1])

    # Add non-meaningful weights

    n_of_rules = len(json_grammar["rules"])
    for r in json_grammar["rules"]:
        r["weight"] = 1/n_of_rules

    return json_grammar

def recursive_state_creation(old_state, list_of_divisions, level, max_level, n_grace_notes):
    ##Create Non Terminal States
    new_states = []  # vector to temporary store states to apply recursion
    new_rules = []
    # append the rule for terminal
    for i in range(n_grace_notes + 2):
        new_rules.append({"head": old_state["name"], "body": ["N{0}".format(i)]})
    ##chose if going on or stop the recursion
    if level > max_level: #stop the recursion
        return new_states, new_rules
    else:
        for d in list_of_divisions:
            rule_body = []
            for n in range(d):
                step= (old_state["name"][1]- old_state["name"][0]) /d
                lower_bound= step* n + old_state["name"][0]
                upper_bound= step* (n+1) + old_state["name"][0]
                new_name = (lower_bound, upper_bound)
                new_states.append({"name": new_name, "type": "non-terminal"})
                ##append the rules
                # create the rule body for the old state for non-terminal
                rule_body.append(new_name)
            new_rules.append({"head": old_state["name"], "body": rule_body}) #append the body to the rule
        ##Call the recursive funcion to each element of new states
        new_new_states = new_states[:] #needed otherwise we are incrementing the list where we loop
        for s in new_states:
            recursion_output = recursive_state_creation(s, list_of_divisions, level+1, max_level, n_grace_notes)
            for s in recursion_output[0]:
                if s not in new_new_states:
                    new_new_states.append(s)
            for r in recursion_output[1]:
                if r not in new_rules:
                    new_rules.append(r)
        return  new_new_states, new_rules


In [6]:
grammar = create_grammar()

Creating generic grammar 


In [7]:
len(grammar["states"])

304

## Vertical Grammar

In [4]:
import string
import os
import json
from fractions import Fraction

def create_vertical_grammar():
    print("Creating generic grammar ")
    path = os.path.join('grammars_test','grammar1.json')

    #Setting the options
    name = "16grammar44"
    namespace = "16grammar44"
    meter_nb_beats = 4
    meter_beat_unit = 4
    levels = 5  #how many time non terminal symbols can be transformed in other non terminal
    grace_notes = 1 #maximum number of grace notes allowed
    list_of_divisions = [2,3] #list_of_division must contain only prime numbers. We can add 11, 13, etc. if we need complex tuplets
    nb_parse_fail = 0

    #create the json
    json_grammar = {
        "name": name,
        "namespace": namespace,
        "meter_nb_beats": meter_nb_beats,
        "meter_beat_unit": meter_beat_unit,
        "parse_failures_ratio": nb_parse_fail,
        "states": [],
        "rules": []

    }
    ## Create States
    # Create Terminal States
    json_grammar["states"].append({"name": "N0", "type": "cont", "nb_grace_notes":0})
    for ii in range(grace_notes + 1):
        json_grammar["states"].append(
            {"name": "N{0}".format(ii + 1), "type": "note", "nb_grace_notes": ii})
    ##Create non terminal states and rules with the recursive funcion
    first_state = {"name": Fraction(1), "type": "non-terminal"}
    recursive_output = recursive_vertical_state_creation(first_state, list_of_divisions, 1, levels, grace_notes)
    json_grammar["states"].append(first_state)
    json_grammar["states"].extend(recursive_output[0])
    json_grammar["rules"].extend(recursive_output[1])

    # Add non-meaningful weights

    n_of_rules = len(json_grammar["rules"])
    for r in json_grammar["rules"]:
        r["weight"] = 1/n_of_rules

    return json_grammar

def recursive_vertical_state_creation(old_state, list_of_divisions, level, max_level, n_grace_notes):
    ##Create Non Terminal States
    new_states = []  # vector to temporary store states to apply recursion
    new_rules = []
    # append the rule for terminal
    for i in range(n_grace_notes + 2):
        new_rules.append({"head": old_state["name"], "body": ["N{0}".format(i)]})
    ##chose if going on or stop the recursion
    if level > max_level: #stop the recursion
        return new_states, new_rules
    else:
        for d in list_of_divisions:
            rule_body = []
            for n in range(d):
                new_name = old_state["name"] /d
                new_states.append({"name": new_name, "type": "non-terminal"})
                ##append the rules
                # create the rule body for the old state for non-terminal
                rule_body.append(new_name)
            new_rules.append({"head": old_state["name"], "body": rule_body}) #append the body to the rule
        ##Call the recursive funcion to each element of new states
        new_new_states = new_states[:] #needed otherwise we are incrementing the list where we loop
        for s in new_states:
            recursion_output = recursive_vertical_state_creation(s, list_of_divisions, level+1, max_level, n_grace_notes)
            for s in recursion_output[0]:
                if s not in new_new_states:
                    new_new_states.append(s)
            for r in recursion_output[1]:
                if r not in new_rules:
                    new_rules.append(r)
        return  new_new_states, new_rules


In [5]:
vert_grammar = create_vertical_grammar()
vert_grammar["rules"]



Creating generic grammar 


[{'body': ['N0'], 'head': Fraction(1, 1), 'weight': 0.010752688172043012},
 {'body': ['N1'], 'head': Fraction(1, 1), 'weight': 0.010752688172043012},
 {'body': ['N2'], 'head': Fraction(1, 1), 'weight': 0.010752688172043012},
 {'body': [Fraction(1, 2), Fraction(1, 2)],
  'head': Fraction(1, 1),
  'weight': 0.010752688172043012},
 {'body': [Fraction(1, 3), Fraction(1, 3), Fraction(1, 3)],
  'head': Fraction(1, 1),
  'weight': 0.010752688172043012},
 {'body': ['N0'], 'head': Fraction(1, 2), 'weight': 0.010752688172043012},
 {'body': ['N1'], 'head': Fraction(1, 2), 'weight': 0.010752688172043012},
 {'body': ['N2'], 'head': Fraction(1, 2), 'weight': 0.010752688172043012},
 {'body': [Fraction(1, 4), Fraction(1, 4)],
  'head': Fraction(1, 2),
  'weight': 0.010752688172043012},
 {'body': [Fraction(1, 6), Fraction(1, 6), Fraction(1, 6)],
  'head': Fraction(1, 2),
  'weight': 0.010752688172043012},
 {'body': ['N0'], 'head': Fraction(1, 4), 'weight': 0.010752688172043012},
 {'body': ['N1'], 'head

In [23]:
import pickle

with open('grammars_test/json/vert_grammar1.pkl', 'wb') as outfile:  
    pickle.dump(vert_grammar, outfile)

### Now print it in a file in the "qparselib" format

In [6]:
#to give an unique integer name to each state

class states_name_mapping:
    def __init__(self):
        self.max_int = 0
        self.mapping = {}
        
    def get_int(self, state_name):
        if state_name in self.mapping:
            return self.mapping[state_name]
        else:
            self.mapping[state_name] = self.max_int
            self.max_int +=1
            return self.mapping[state_name]
    
    def get_mapping(self):
        return self.mapping
            

In [7]:
def print_grammar(grammar, name):
    mapping = states_name_mapping()
    
    str_out = "//Grammar\n"
    str_out = "{0}{1}\n".format(str_out, "[penalty]")
    for i, r in enumerate(grammar["rules"]):
        str_out = "{0}{1}\n".format(str_out, print_rule(r,mapping))
#         print("retrieved rule " + str(i))
    str_out += "\n\n"
    #save the file
    with open(os.path.join("grammars_test",name), "w") as f:
        f.write(str_out)
    #return the grammar for later evaluation
    return mapping
            
def print_rule(rule, mapping):            
    body_str = "("
    if len(rule["body"])== 1: #if it's a terminal
        body_str = "{0} {1} ".format(body_str, rule["body"][0][1:])
    else: # if it's a non terminal
        for s in rule["body"]:
            body_str = "{0} {1} ".format(body_str, mapping.get_int(s))
    body_str = body_str + ")"
    return "{0} {1} {2}".format(mapping.get_int(rule["head"]), body_str, rule["weight"])

In [19]:
mapping = print_grammar(grammar)

mapping.get_mapping()

{(Fraction(0, 1), Fraction(1, 81)): 190,
 (Fraction(0, 1), Fraction(1, 54)): 62,
 (Fraction(0, 1), Fraction(1, 36)): 26,
 (Fraction(0, 1), Fraction(1, 27)): 187,
 (Fraction(0, 1), Fraction(1, 24)): 18,
 (Fraction(0, 1), Fraction(1, 18)): 59,
 (Fraction(0, 1), Fraction(1, 16)): 16,
 (Fraction(0, 1), Fraction(1, 12)): 13,
 (Fraction(0, 1), Fraction(1, 9)): 184,
 (Fraction(0, 1), Fraction(1, 8)): 11,
 (Fraction(0, 1), Fraction(1, 6)): 8,
 (Fraction(0, 1), Fraction(1, 4)): 6,
 (Fraction(0, 1), Fraction(1, 3)): 3,
 (Fraction(0, 1), Fraction(1, 2)): 1,
 (Fraction(0, 1), Fraction(1, 1)): 0,
 (Fraction(1, 81), Fraction(2, 81)): 191,
 (Fraction(1, 54), Fraction(1, 27)): 63,
 (Fraction(2, 81), Fraction(1, 27)): 192,
 (Fraction(1, 36), Fraction(1, 18)): 27,
 (Fraction(1, 27), Fraction(4, 81)): 193,
 (Fraction(1, 27), Fraction(1, 18)): 64,
 (Fraction(1, 27), Fraction(2, 27)): 188,
 (Fraction(1, 24), Fraction(1, 12)): 19,
 (Fraction(4, 81), Fraction(5, 81)): 194,
 (Fraction(1, 18), Fraction(2, 27))

In [16]:
#print the vertical grammar
mapping = print_grammar(vert_grammar,"vert_grammar1.txt")

mapping.get_mapping()

{Fraction(1, 243): 20,
 Fraction(1, 162): 16,
 Fraction(1, 108): 13,
 Fraction(1, 81): 19,
 Fraction(1, 72): 11,
 Fraction(1, 54): 15,
 Fraction(1, 48): 10,
 Fraction(1, 36): 12,
 Fraction(1, 32): 9,
 Fraction(1, 27): 18,
 Fraction(1, 24): 8,
 Fraction(1, 18): 14,
 Fraction(1, 16): 7,
 Fraction(1, 12): 6,
 Fraction(1, 9): 17,
 Fraction(1, 8): 5,
 Fraction(1, 6): 4,
 Fraction(1, 4): 3,
 Fraction(1, 3): 2,
 Fraction(1, 2): 1,
 Fraction(1, 1): 0}